In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import Adam
from torchvision import datasets

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
X_train = torch.tensor([[[[1,2,3,4],[2,3,4,5],
                          [5,6,7,8],[1,3,4,5]]],
                        [[[-1,2,3,-4],[2,-3,4,5],
                          [-5,6,-7,8],[-1,-3,-4,-5]]]]
                       ).to(device).float()

X_train /= 8
y_train = torch.tensor([0,1]).to(device).float()

In [4]:
def get_model():
  model = nn.Sequential(
      nn.Conv2d(1, 1, kernel_size=3),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(1, 1),
      nn.Sigmoid()
  ).to(device)

  loss_fn = nn.BCELoss()
  optimizer = Adam(model.parameters(), lr=1e-3)

  return model, loss_fn, optimizer

In [6]:
!pip install torch_summary
from torchsummary import summary

model, loss_fn, optimizer = get_model()
summary(model, X_train)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [7]:
def train_batch(x, y, model, opt, loss_fn):
  model.train()

  pred = model(x)
  batch_loss = loss_fn(pred.squeeze(0), y)
  batch_loss.backward()
  opt.step()
  opt.zero_grad()

  return batch_loss.item()

In [8]:
trn_dl = DataLoader(TensorDataset(X_train, y_train))

In [9]:
for epoch in range(2000):
  for ix, batch in enumerate(iter(trn_dl)):
    x, y = batch
    batch_loss = train_batch(x, y, model, optimizer, loss_fn)

In [10]:
model(X_train[:1])

tensor([[0.1439]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [11]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]